In [ ]:
cd

In [1]:
from DemoServerDB import ConnectDatabase
from mapping import doc

In [2]:
temp = ConnectDatabase()

In [3]:
temp.conn

<connection object at 0x7efff5bd06e0; dsn: 'dbname=postgresdemotwo user=clientdemo password=xxxxxxxxxxx host=postgresdemotwo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com port=5432 sslmode=require', closed: 0>

In [4]:
doc['column_map']['TRANSACTION_MASTER']

{'category': 'category',
 'cust_id': 'id',
 'product_id': 'prod_id',
 'quantity': 'purchasequantity',
 'timestamp': 'date'}

In [5]:
query = 'SELECT "%s","%s","%s" FROM "%s"'%(doc['column_map']['TRANSACTION_MASTER']['cust_id'],
                                           doc['column_map']['TRANSACTION_MASTER']['product_id'],
                                           doc['column_map']['TRANSACTION_MASTER']['timestamp'],
                                           doc['table_map']['TRANSACTION_MASTER'])

In [6]:
print query

SELECT "id","prod_id","date" FROM "trans"


In [7]:
temp.conn.rollback()

In [8]:
temp.cur.execute(query)

In [9]:
col_names = [desc[0] for desc in temp.cur.description]

In [10]:
col_names

['id', 'prod_id', 'date']

In [11]:
import pandas as pd
import numpy as np
import datetime as dt

In [12]:
trans_table = pd.DataFrame(temp.cur.fetchall(), columns=col_names)

In [13]:
trans_table.head()

,id,prod_id,date
0,2255653295,5824 - 17040 - 18 - OZ,2012-04-17
1,2255653295,416 - 56124 - 1.85 - OZ,2012-04-17
2,2255653295,2633 - 16048 - 4 - RL,2012-04-17
3,2255653295,2613 - 8583 - 27 - CT,2012-04-17
4,2255653295,3611 - 6303 - 101.4 - OZ,2012-04-17


In [14]:
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'},inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['product_id']:'product_id'},inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['timestamp']:'timestamp'},inplace=True)

In [15]:
trans_table.head()

,cust_id,product_id,timestamp
0,2255653295,5824 - 17040 - 18 - OZ,2012-04-17
1,2255653295,416 - 56124 - 1.85 - OZ,2012-04-17
2,2255653295,2633 - 16048 - 4 - RL,2012-04-17
3,2255653295,2613 - 8583 - 27 - CT,2012-04-17
4,2255653295,3611 - 6303 - 101.4 - OZ,2012-04-17


In [16]:
trans_table['timestamp'] = pd.to_datetime(trans_table['timestamp'])

In [17]:
trans_table.shape

(71932603, 3)

In [ ]:
len(trans_table.cust_id.unique())

In [ ]:
len(trans_table.product_id.unique())

In [18]:
group_data = trans_table.groupby(['cust_id', 'product_id']).size().reset_index().rename(columns={0:'counter'})

In [19]:
group_data.head()

,cust_id,product_id,counter
0,100010021,1205 - 17292 - 5 - OZ,2
1,100010021,1703 - 10882 - 32 - OZ,1
2,100010021,1703 - 19934 - 26 - OZ,1
3,100010021,1703 - 6608 - 32 - OZ,1
4,100010021,1704 - 27327 - 20 - OZ,1


In [ ]:
group_data.shape

In [20]:
repeated_data = group_data.query('counter != 1')

In [21]:
repeated_data.head()

,cust_id,product_id,counter
0,100010021,1205 - 17292 - 5 - OZ,2
16,100010021,1905 - 15704 - 18 - OZ,3
17,100010021,1905 - 16728 - 18 - OZ,2
18,100010021,1908 - 9189 - 28 - OZ,2
21,100010021,2108 - 15704 - 64 - OZ,12


In [22]:
repeated_data.shape

(9935544, 3)

In [23]:
len(repeated_data.cust_id.unique())

62195

In [24]:
len(repeated_data.product_id.unique())

131042

In [ ]:
repeated_data.head()

In [ ]:
trans_table.head()

In [27]:
xyz = pd.merge(trans_table, repeated_data, on=['cust_id', 'product_id'], how='right')

In [28]:
xyz.shape

(54175208, 4)

In [ ]:
xyz.head()

In [29]:
len(xyz.product_id.unique())

131042

In [ ]:
def x(frame):
    frame['time'] = frame['timestamp'].max() - frame['timestamp'].min()
    return frame

In [30]:
dates = xyz.groupby(['cust_id','product_id'], group_keys=False).transform(lambda x : x.timestamp.max() - x.timestamp.min())

AttributeError: ("'Series' object has no attribute 'timestamp'", u'occurred at index timestamp')

In [ ]:
dates.head()

In [ ]:
def z(frame):
    frame['time'] = frame['timestamp'].diff()
    return frame

In [ ]:
dates2 = xyz.groupby(['cust_id','product_id'], group_keys=False).apply(z)

In [ ]:
dates2.head()

In [ ]:
dates.shape

In [ ]:
len(dates.cust_id.unique())

In [ ]:
len(dates.product_id.unique())

In [ ]:
dates.reset_index(drop=True,inplace=True)

In [ ]:
dates.head()

In [ ]:
cust_repur = dates

In [ ]:
cust_repur.head()

In [ ]:
cust_repur.shape

In [ ]:
cust_repur.reset_index(drop=True, inplace=True)

In [ ]:
cust_repur.head()

In [ ]:
cust_repur.shape

In [ ]:
list_of_indexes = [np.argmax(g['timestamp']) for l, g in cust_repur.groupby(['cust_id','product_id'])]

In [ ]:
cust_repur_original = cust_repur.ix[list_of_indexes]

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original.reset_index(drop=True,inplace=True)

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original['days_int'] = cust_repur_original['time'].dt.days

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original['repurchase_time'] = (cust_repur_original['days_int']-1) / (cust_repur_original['counter']-1)

In [ ]:
cust_repur_original.head()

In [ ]:
len(cust_repur_original.product_id.unique())

In [ ]:
abc = pd.pivot_table(cust_repur_original, index=['cust_id'], columns=['product_id'], values=['repurchase_time'], aggfunc='first')

In [ ]:
abc.shape

In [ ]:
abc.head()

In [ ]:
type(abc)

In [ ]:
mi = abc.columns

In [ ]:
mi.tolist()

In [ ]:
ind = pd.Index([e[1] for e in mi.tolist()])

In [ ]:
abc.columns = ind

In [ ]:
abc.head()

In [ ]:
products = pd.DataFrame(abc.columns)

In [ ]:
products.rename(columns={0:'product_id'}, inplace=True)

In [ ]:
art = abc[products.product_id[products.index]].sum() / abc[products.product_id[products.index]].count()

In [ ]:
type(art)

In [ ]:
art = art.to_frame()

In [ ]:
art.reset_index(inplace=True)

In [ ]:
art.rename(columns={'index':'product_id'}, inplace=True)

In [ ]:
art.rename(columns={0:'average_repurchase_time'}, inplace=True)

In [ ]:
art

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original['time_diff'] = dt.date.today() - cust_repur_original["timestamp"].dt.date

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original.tail()

In [ ]:
cust_repur_original.counter = cust_repur_original.counter.fillna(0)

In [ ]:
cust_repur_original.head()

In [ ]:
art.tail()

In [ ]:
cust_repur_original_final = cust_repur_original.merge(art, on=['product_id'], how='left')

In [ ]:
cust_repur_original_final.head()

In [ ]:
cust_repur_original.shape

In [ ]:
cust_repur_original_final.shape

In [ ]:
cust_repur_original_final.repurchase_time.fillna(cust_repur_original_final.average_repurchase_time, inplace=True)

In [ ]:
cust_repur_original_final.head()

In [ ]:
cust_repur_original_final['time_diff_int'] = cust_repur_original_final['time_diff'].dt.days

In [ ]:
cust_repur_original_final.head()

In [ ]:
cust_repur_original['depriotarize'] = ""

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original_final.repurchase_time = cust_repur_original_final.repurchase_time.fillna(0)

In [ ]:
cust_repur_original_final.head()

In [ ]:
type(cust_repur_original.time_diff[1])

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original_final['depre'] = ""

In [ ]:
if cust_repur_original_final["time_diff_int"].all() > cust_repur_original_final["repurchase_time"].all():
    cust_repur_original["depre"] = "Recommend"
else:
    cust_repur_original["depre"] = "Do_not_Recommend"

In [ ]:
cust_repur_original_final.head()

In [ ]:
cust_repur_original.head()

In [ ]:
del cust_repur_original['timestamp']

In [ ]:
del cust_repur_original['counter']

In [ ]:
del cust_repur_original['time']

In [ ]:
del cust_repur_original['days_int']

In [ ]:
del cust_repur_original['repurchase_time']

In [ ]:
del cust_repur_original['time_diff']

In [ ]:
del cust_repur_original['time_diff_int']

In [ ]:
cust_repur_original.head()